In [ ]:
# TODO: shift points due to extension
# TODO: fix LM plane masks

In [96]:
import numpy as np
import tifffile
import napari
import os
from skimage.measure import regionprops, label
from scipy.spatial.transform import Rotation

from scripts.config_model import save_experiment_config, tree
from scripts.sample_db import SampleDB
from scripts.utils.image_utils import load_tiff_as_hyperstack, save_array_as_hyperstack_tiff

In [99]:
# Function to apply transformation
def apply_transformation(points, transform):
    rotation = Rotation.from_matrix(transform[:3, :3])
    translation = transform[:3, 3]
    return rotation.apply(points) + translation


# Function to check if a point is within the 3D mask
def is_within_mask(point, mask):
    x, y, z = np.round(point).astype(int)
    if 0 <= x < mask.shape[0] and 0 <= y < mask.shape[1] and 0 <= z < mask.shape[2]:
        return mask[x, y, z]
    return False


In [100]:
# Step 1: Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)

# Step 2: Load experiment configuration
sample_id = '20220426_RM0008_130hpf_fP1_f3'
exp = sample_db.get_sample(sample_id)
print(exp.sample.id)


20220426_RM0008_130hpf_fP1_f3


In [117]:
old_t = t_lm_trial_to_lm_stack

In [102]:
# Step 3: Load images and transformation matrices
#LM stack raw
lm_stack_raw_path = os.path.join(exp.paths.anatomy_path, "raw", f"{sample_id}_anatomyGFRF_001_.tif")
lm_stack_raw = load_tiff_as_hyperstack(lm_stack_raw_path, n_channels=2)

#LM stack bw (Bigwarp)
lm_stack_bw_path = os.path.join(exp.paths.anatomy_path, "processed", f"upsampled_{sample_id}_anatomyGFRF_001_.tif")
lm_stack_bw = load_tiff_as_hyperstack(lm_stack_bw_path)[:,0]

#LM trials images
processed_folder = os.path.join(exp.paths.trials_path, "processed")

lm_trials_images_path = os.path.join(processed_folder, f"sum_raw_trials_{exp.sample.id}.tif")
lm_trials_images =tifffile.imread(lm_trials_images_path)
plane_nr = 6
lm_trial = lm_trials_images[plane_nr,-1]
print(lm_trial.shape)

#LM trials masks
lm_planes_masks_path = os.path.join(exp.paths.trials_path,"masks",f"masks_{exp.sample.id}_params_cp_-1-ft_0.4-st_0.01-resample_True_augment=False.tif")
lm_planes_masks = tifffile.imread(lm_planes_masks_path)

lm_mask = lm_planes_masks[plane_nr,-1]
print(lm_mask.shape)

lm_plane_props = regionprops(label(lm_mask))
lm_plane_centroids =  np.array([prop.centroid for prop in lm_plane_props])

#EM warped stack
em_warped_stack_path = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_em_stack_cropped_woResin_rough_rotated_to_LM_fov_lm_res_em.tif')
em_warped_stack = tifffile.imread(em_warped_stack_path)

em_warped_mask_path = os.path.join(exp.paths.em_path, '20220426_RM0008_130hpf_fP1_f3_em_stack_cropped_woResin_rough_rotated_to_LM_fov_lm_res_em_mask_filtered.tif')
em_warped_mask = tifffile.imread(em_warped_stack_path)

em_stack_lmov_path = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\CLEM_Analyses\CLEM_20220426_RM0008_130hpf_fP1_f3\pycpd\em_stack_channel 1_xfm_0_flfov_lmresolution.tif"
em_stack_lmov = tifffile.imread(em_stack_lmov_path)



\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\raw\20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif loaded. Shape: (2, 150, 512, 512)
\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\anatomy\processed\upsampled_20220426_RM0008_130hpf_fP1_f3_anatomyGFRF_001_.tif loaded. Shape: (375, 2, 512, 512)
(256, 512)
(256, 512)


In [104]:
# Visualize
#viewer = napari.Viewer()
viewer.add_image(lm_stack_raw)
viewer.add_image(lm_stack_bw)
viewer.add_image(lm_trial)
viewer.add_image(lm_mask, blending='additive', colormap='viridis')
viewer.add_image(em_warped_stack)
viewer.add_image(em_stack_lmov)



<Image layer 'em_stack_lmov' at 0x23fc3d64dc0>

In [109]:
extended_stack = np.pad(lm_stack_bw, ((0, 0), (20, 20), (20, 20)), mode='minimum', stat_length=50)
viewer.add_image(extended_stack)

<Image layer 'extended_stack' at 0x23fb5e4d870>

In [115]:
# Test transformation matrix for plane 0, last trial
t_lm_trial_to_lm_stack = np.load(r"D:\montruth\PycharmProjects\samplePy\notebooks\registration_tform_lm_plane06_lm_stack.npy")
print(t_lm_trial_to_lm_stack)

[[ 1.05036699e+00 -1.68973879e-01  1.44150470e-02  4.41737571e+01]
 [ 1.69008631e-01  1.05045931e+00 -1.45009191e-03  1.52613233e+02]
 [-1.40017122e-02  3.72134407e-03  1.06387071e+00  1.05388168e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [110]:
lm_plane_centroids_3d_original = np.hstack((np.zeros(shape=(lm_plane_centroids.shape[0],1)), lm_plane_centroids[::-1]))

viewer.add_points(lm_plane_centroids_3d_original)

lm_plane_centroids_3d_original

array([[  0.        , 155.06741573, 333.74157303],
       [  0.        , 153.60305344, 284.19847328],
       [  0.        , 142.1375    , 356.65625   ],
       [  0.        , 139.48387097, 287.56451613],
       [  0.        , 137.83802817, 259.93309859],
       [  0.        , 129.3254717 , 179.37264151],
       [  0.        , 122.66019417, 253.00485437],
       [  0.        , 117.88622754, 200.11976048],
       [  0.        , 117.60580913, 170.09958506],
       [  0.        , 115.97297297, 267.64189189],
       [  0.        , 112.4       , 376.45555556],
       [  0.        , 106.13934426, 275.76229508],
       [  0.        , 101.76190476, 415.02857143],
       [  0.        , 104.34146341, 256.67479675],
       [  0.        , 104.40856031, 324.03501946],
       [  0.        , 103.93140794,  58.10108303],
       [  0.        ,  98.49621212, 308.97727273],
       [  0.        ,  98.11715481, 215.10878661],
       [  0.        ,  99.22972973, 367.69369369],
       [  0.        ,  95.79545

In [111]:
lm_plane_centroids_3d_offset = lm_plane_centroids_3d_original+[0,-20,-20]
viewer.add_points(lm_plane_centroids_3d_offset)
lm_plane_centroids_3d_offset

array([[ 0.00000000e+00,  1.35067416e+02,  3.13741573e+02],
       [ 0.00000000e+00,  1.33603053e+02,  2.64198473e+02],
       [ 0.00000000e+00,  1.22137500e+02,  3.36656250e+02],
       [ 0.00000000e+00,  1.19483871e+02,  2.67564516e+02],
       [ 0.00000000e+00,  1.17838028e+02,  2.39933099e+02],
       [ 0.00000000e+00,  1.09325472e+02,  1.59372642e+02],
       [ 0.00000000e+00,  1.02660194e+02,  2.33004854e+02],
       [ 0.00000000e+00,  9.78862275e+01,  1.80119760e+02],
       [ 0.00000000e+00,  9.76058091e+01,  1.50099585e+02],
       [ 0.00000000e+00,  9.59729730e+01,  2.47641892e+02],
       [ 0.00000000e+00,  9.24000000e+01,  3.56455556e+02],
       [ 0.00000000e+00,  8.61393443e+01,  2.55762295e+02],
       [ 0.00000000e+00,  8.17619048e+01,  3.95028571e+02],
       [ 0.00000000e+00,  8.43414634e+01,  2.36674797e+02],
       [ 0.00000000e+00,  8.44085603e+01,  3.04035019e+02],
       [ 0.00000000e+00,  8.39314079e+01,  3.81010830e+01],
       [ 0.00000000e+00,  7.84962121e+01

In [112]:
transformed_lm_centroids_original = apply_transformation(lm_plane_centroids_3d_original, t_lm_trial_to_lm_stack)
viewer.add_points(transformed_lm_centroids_original)
transformed_lm_centroids_original

array([[ 22.79243658, 305.6517301 , 343.82760727],
       [ 22.36433251, 304.19517959, 294.2861269 ],
       [ 25.20352087, 292.89940124, 366.71493831],
       [ 24.70594426, 290.26480885, 297.62426762],
       [ 24.60202997, 288.63436397, 269.99213262],
       [ 24.90311528, 280.21615861, 189.42232419],
       [ 26.9773807 , 273.65701055, 263.03484823],
       [ 27.04260682, 268.93413948, 210.14520702],
       [ 26.68449525, 268.65037279, 180.1271991 ],
       [ 28.26183315, 267.06229271, 277.65749072],
       [ 30.30629501, 263.56257074, 386.45432733],
       [ 29.97038011, 257.36152896, 285.75793834],
       [ 32.55521479, 253.07523758, 425.00306048],
       [ 30.00609406, 255.58308635, 266.66865248],
       [ 30.90105856, 255.66518158, 334.02291295],
       [ 27.40231647, 255.13164329,  68.11210017],
       [ 31.66014274, 249.8280125 , 318.95497195],
       [ 30.45942573, 249.43185717, 225.09423623],
       [ 32.33047654, 250.56561479, 377.6675152 ],
       [ 32.66712417, 247.17317

In [113]:
transformed_lm_centroids_offset = apply_transformation(lm_plane_centroids_3d_offset, t_lm_trial_to_lm_stack)
viewer.add_points(transformed_lm_centroids_offset)
transformed_lm_centroids_offset

array([[ 25.77627232, 285.91334095, 323.79032407],
       [ 25.34816825, 284.45679043, 274.2488437 ],
       [ 28.18735661, 273.16101209, 346.67765512],
       [ 27.68978   , 270.5264197 , 277.58698442],
       [ 27.58586571, 268.89597482, 249.95484942],
       [ 27.88695102, 260.47776946, 169.385041  ],
       [ 29.96121644, 253.9186214 , 242.99756503],
       [ 30.02644256, 249.19575032, 190.10792382],
       [ 29.66833099, 248.91198364, 160.08991591],
       [ 31.24566889, 247.32390356, 257.62020752],
       [ 33.29013075, 243.82418159, 366.41704414],
       [ 32.95421585, 237.62313981, 265.72065514],
       [ 35.53905053, 233.33684843, 404.96577728],
       [ 32.9899298 , 235.8446972 , 246.63136928],
       [ 33.8848943 , 235.92679242, 313.98562975],
       [ 30.38615221, 235.39325413,  48.07481697],
       [ 34.64397848, 230.08962335, 298.91768875],
       [ 33.44326147, 229.69346802, 205.05695303],
       [ 35.31431228, 230.82722564, 357.630232  ],
       [ 35.65095991, 227.43479

In [ ]:
transformed_lm_centroids = apply_transformation(lm_plane_centroids_3d, t_lm_trial_to_lm_stack)
transformed_lm_centroids_offset = transformed_lm_centroids + [0,-20,-20]

In [67]:
viewer.add_points(lm_plane_centroids_3d, name="lm_plane_centroids_3d", face_color = "red")
viewer.add_points(transformed_lm_centroids, name="transformed_lm_centroids", face_color = "blue")

<Points layer 'transformed_lm_centroids [1]' at 0x23fa737ca30>

In [64]:
viewer.add_points(lm_plane_centroids_3d_offset, name="lm_plane_centroids_3d_offset", face_color = "magenta")
viewer.add_points(transformed_lm_centroids_offset, name="transformed_lm_centroids_offset", face_color = "cyan")

<Points layer 'transformed_lm_centroids_offset' at 0x23fa8099a20>

In [119]:
# Current problematic code
lm_plane_centroids_3d_original = np.hstack((np.zeros(shape=(lm_plane_centroids.shape[0],1)), lm_plane_centroids[::-1]))

# Fix the coordinate order (z,y,x)
lm_plane_centroids_3d = np.column_stack((
    np.zeros(len(lm_plane_centroids)),  # z coordinate
    lm_plane_centroids[:, 0],           # y coordinate
    lm_plane_centroids[:, 1]            # x coordinate
))

def apply_transformation(points, transform):
    # Convert to homogeneous coordinates
    homogeneous_points = np.column_stack((points, np.ones(len(points))))
    # Apply transformation
    transformed_points = homogeneous_points @ transform.T
    # Return to 3D coordinates
    return transformed_points[:, :3]


transformed_points = apply_transformation(lm_plane_centroids_3d_original, t_lm_trial_to_lm_stack)
t_new = t_lm_trial_to_lm_stack
t_new[0, 3] -= 20  # x translation
t_new[1, 3] -= 20  # y translation
transformed_points_new = apply_transformation(lm_plane_centroids_3d_original, t_new)


viewer = napari.Viewer()
viewer.add_image(lm_trial, name='Original Image')
viewer.add_image(lm_mask, blending='additive', colormap='viridis')
viewer.add_points(lm_plane_centroids_3d, name='Original Points', size=5)
viewer.add_points(transformed_points, name='Transformed Points', size=5, face_color='red')
viewer.add_points(transformed_points_new, name='transformed_points_new', size=5, face_color='blue')
viewer.add_image(extended_stack)
viewer.add_image(lm_stack_raw, name="lm_stack")



D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)
D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Image layer 'lm_stack' at 0x23f7d035150>

In [120]:
viewer.add_image(lm_stack_raw)

<Image layer 'lm_stack_raw' at 0x23fc4001a50>

In [79]:
viewer.add_image(lm_mask, blending='additive', colormap='viridis')

<Image layer 'lm_mask' at 0x23fbb61b9d0>

In [90]:
selected_point = np.array(viewer.layers["Points"].data)[0,1:]
print(selected_point)

[  9.         155.69095428 110.07069617]


In [91]:
selected_point_reshaped = selected_point.reshape(1, -1)
print(selected_point_reshaped)

[[  9.         155.69095428 110.07069617]]


In [94]:
transformed_point = apply_transformation(selected_point_reshaped, t_lm_trial_to_lm_stack)
print(transformed_point)
viewer.add_points(transformed_point,face_color='blue')

[[ 15.77733643 308.33275897 120.75596002]]


<Points layer 'transformed_point [1]' at 0x23fc05bacb0>

In [123]:
from skimage.transform import warp
from skimage.transform import SimilarityTransform

# Create transform object
tform = SimilarityTransform(matrix=t_new)

# Apply transformation to plane or mask
transformed_image = warp(lm_trial, tform.inverse, output_shape=lm_stack_bw.shape)


In [125]:
import cv2

# Extract 2D affine matrix (first two rows of 3D transformation)
affine_matrix = transformation_matrix[:2]

# Apply transformation
transformed_image = cv2.warpAffine(
    lm_trial,
    affine_matrix, 
    dsize=(lm_stack_bw.shape[2], lm_stack_bw.shape[1])
)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\imgwarp.cpp:2834: error: (-215:Assertion failed) (M0.type() == CV_32F || M0.type() == CV_64F) && M0.rows == 2 && M0.cols == 3 in function 'cv::warpAffine'


In [124]:
viewer.add_image(transformed_image, name='transformed_image')

<Image layer 'transformed_image' at 0x23faa2585e0>

In [ ]:
#######################################################

In [61]:
# Transformation matrices
t_lm_trial_to_lm_stack_path = os.path.join(processed_folder, f"registration_tform_lm_plane0{plane_nr}_lm_stack.npy")
t_lm_trial_to_lm_stack = np.load(t_lm_trial_to_lm_stack_path)

In [62]:
t_lm_trial_to_lm_stack.shape

(4, 4)

In [63]:
transformed_lm_centroids = apply_transformation(lm_plane_centroids_3d, t_lm_trial_to_lm_stack)

In [73]:
#Visualize
viewer = napari.Viewer()
viewer.add_image(lm_trial, name="lm_trial")
viewer.add_labels(label(lm_mask), name="lm_mask")
viewer.add_image(lm_stack_raw, name="lm_stack")
viewer.add_image(em_warped_stack, name="em_warped_stack")

#viewer.add_points(lm_plane_centroids, name="lm_plane_centroids")
#viewer.add_points(lm_plane_centroids_3d, name="lm_plane_centroids_3d")
viewer.add_points(transformed_lm_centroids, name="transformed_lm_centroids")

<Points layer 'transformed_lm_centroids' at 0x15c437387d0>

In [ ]:
#######################################################

In [1]:
import numpy as np
import pandas as pd
import napari
from scipy.spatial.transform import Rotation
import tifffile

# Function to apply transformation
def apply_transformation(points, transform):
    rotation = Rotation.from_matrix(transform[:3, :3])
    translation = transform[:3, 3]
    return rotation.apply(points) + translation


# Function to check if a point is within the 3D mask
def is_within_mask(point, mask):
    x, y, z = np.round(point).astype(int)
    if 0 <= x < mask.shape[0] and 0 <= y < mask.shape[1] and 0 <= z < mask.shape[2]:
        return mask[x, y, z]
    return False


# Load transformation matrix for lm trials to lm_stack
t_lm_trials_to_lm_stack = np.load("t_lm_trials_to_lm_stack.npy")

# Load centroids of lm plane
lm_trials_centroids = np.load("lm_trials_centroids.npy")

# Load em warped mask
em_warped_mask = tifffile.imread("em_warped_mask.tif")

# Apply transformations to lm_plane_centroids to fit lm_stak and to ultimately fit to em_stack 

centroids_in_lm_stack = apply_transformation(lm_plane_centroids, lm_to_lm_stack)
centroids_in_em_stack = apply_transformation(centroids_in_lm_stack, lm_stack_to_em_stack)


# Identify  centroids within the 3D mask

centroids_in_mask = [point for point in centroids_in_em_stack if is_within_mask(point, em_mask)]


# Validate by visual inspection in napari (Found cells in green)
# Create Napari viewer
viewer = napari.Viewer()

# Add layers to the viewer
viewer.add_image(em_mask, name='EM Mask')
viewer.add_points(points_layer, size=5, face_color='green', name='Found Cells')

# Run Napari viewer
napari.run()


FileNotFoundError: [Errno 2] No such file or directory: 't_lm_trials_to_lm_stack.npy'